In [2]:
#定义库
import torch
from torch import nn
import numpy as np

In [7]:
text = ['hey how are you','good i am fine','have a nice day']#创建一个三个字符串的列表
 
# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))#''.join(text)去除标点符号 set()是去除重复的字母
print(chars)
# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))
print(int2char)
# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

{'n', 'r', 'g', 'y', ' ', 'a', 'w', 'u', 'm', 'v', 'i', 'd', 'h', 'f', 'c', 'o', 'e'}
{0: 'n', 1: 'r', 2: 'g', 3: 'y', 4: ' ', 5: 'a', 6: 'w', 7: 'u', 8: 'm', 9: 'v', 10: 'i', 11: 'd', 12: 'h', 13: 'f', 14: 'c', 15: 'o', 16: 'e'}
{'n': 0, 'r': 1, 'g': 2, 'y': 3, ' ': 4, 'a': 5, 'w': 6, 'u': 7, 'm': 8, 'v': 9, 'i': 10, 'd': 11, 'h': 12, 'f': 13, 'c': 14, 'o': 15, 'e': 16}


In [15]:
# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))#key=len 指定按字符串长度进行比较 让三个字符串进行比较
 
# Padding
 
# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '#让3个字符串的长度统一

['hey how are you', 'good i am fine ', 'have a nice day']


In [16]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []
 
for i in range(len(text)):
    # Remove last character for input sequence
  input_seq.append(text[i][:-1])
    
    # Remove first character for target sequence
  target_seq.append(text[i][1:])
  print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [17]:
#定义库
import torch
from torch import nn
import numpy as np

text = ['hey how are you', 'good i am fine', 'have a nice day']  # 创建一个三个字符串的列表

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))  # ''.join(text)去除标点符号 set()是去除重复的字母
#print(chars)
# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))
#print(int2char)
# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
#print(char2int)

# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))  # key=len 指定按字符串长度进行比较 让三个字符串进行比较

# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
    while len(text[i]) < maxlen:
        text[i] += ' '  # 让3个字符串的长度统一
#print(text)
#字符数算上了空格，只有第二句的字符数是14

# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])

    # Remove first character for target sequence
    target_seq.append(text[i][1:])
    #print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))
#由字母转为数字
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]#character表示字符的意思
#print(input_seq)
#print(target_seq)#每次运行char2int[character]会发生变化
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)
#print(batch_size)


def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)#初始化

    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features#三个句子中的一个句子中的一个字母为1 一个地方对应一个矩阵
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)
#-------------------------------------------------------------------------------------------------------------------------
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")


class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
#model.to(device)
# Define hyperparameters
n_epochs = 100
lr=0.01
# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()#交叉熵误差
optimizer = torch.optim.Adam(model.parameters(), lr=lr)#优化器

# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad()  # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward()  # Does backpropagation and calculates gradients
    optimizer.step()  # Updates the weights accordingly

    if epoch % 10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))


# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)

    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden


# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval()  # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]#将起始字符串拆分为字符列表
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

GPU is available
Epoch: 10/100............. Loss: 2.3907
Epoch: 20/100............. Loss: 2.1222
Epoch: 30/100............. Loss: 1.7358
Epoch: 40/100............. Loss: 1.3280
Epoch: 50/100............. Loss: 0.9666
Epoch: 60/100............. Loss: 0.6800
Epoch: 70/100............. Loss: 0.4695
Epoch: 80/100............. Loss: 0.3264
Epoch: 90/100............. Loss: 0.2326
Epoch: 100/100............. Loss: 0.1726


In [18]:
print(sample(model, 15, 'hey'))

hey how are you
